In [1]:
import numpy as np

## Layer Class

In [2]:
class Layer():
    def __init__(self, i, errors):
        self.layers = i
        self.errs = errors
        
    
    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X
    
    def backward(self):
        self.grad = self.errs.backward()
        i = len(self.layers) - 1
        
        while i >= 0:
            self.grad = self.layers[i].backward(self.grad)
            i -= 1
        
    def _errs(self, X, y):
        return self.errs.forward(self.forward(X), y)

## Linear Class

In [3]:
class Linear(Layer):
    def __init__(self, X, y):
        self.w = np.random.rand(X, y) * 0.001
        self.b = np.zeros(y)
    
    def forward(self, X):
        self.X = X
        self.y = (X @ self.w) + self.b
        
        return self.y
    
    def backward(self, grad_X):
        self.dw = np.matmulmatmul(self.X[:,:,None], gradient_X[:,None,:]).mean(axis=0)
        self.db = grad_X.mean(axis=0)
        
        return grad_X @ self.w.T
    
    def update_w_b(self, lr = 0.001):
        self.w -= lr * self.dw
        self.b -= lr * self.db

## Sigmoid Function

In [4]:
class Logistic_Sigmoid(Layer):
    def __init__(self):
        self.y = None
    
    def forward(self, X):
        self.y = 1.0 / (1.0 + np.exp(-X))
        
        return self.y
    
    def backward(self, grad_X):
        return self.y * (1 - self.y) * grad_X

## Hyperbolic Tangent Function

In [5]:
class Hyperbolic_Tangent(Layer):
    def __init__(self):
        self.y = None
    
    def forward(self, X):
        self.y = np.tanh(X)
        
        return self.y
    
    def backward(self, grad_X):
        return (1 - np.square(self.outputs)) * gradient_X

## Softmax

In [6]:
class Softmax(Layer):
    def __init__(self):
        self.num = None
        self.dim = None
        self.y = None
    
    def forward(self, X):
        self.num = np.exp(X - np.max(X))
        
        return self.num / np.sum(self.num, axis=0, keepdims=True)
    
    def backward(self, probs, bp_err):
        self.dim = probs.shape[1]
        self.y = np.empty(probs.shape)
        
        for j in range(self.dim):
            d_prob_over_xj = - (probs * probs[:,[j]])  # i.e. prob_k * prob_j, no matter k==j or not
            d_prob_over_xj[:,j] += probs[:,j]   # i.e. when k==j, +prob_j
            self.y[:,j] = np.sum(bp_err * d_prob_over_xj, axis=1)
        return self.y

## Cross-Entropy Loss

In [7]:
class Cross_Entropy_Loss(Layer):
    def __init__(self):
        pass
    
    def forward(self, y):
        return -y * np.log(self.y)

    def backward(self, y):
        return y - self.y

## Sequential Class

In [8]:
class Sequential(Layer):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
        self.w = None
        self.b = None
        
        self.layers = [Linear(X, y), Logistic_Sigmoid(), Linear(X, y), Logistic_Sigmoid()]
        self.model = Layer(self.layers, Cross_Entropy_Loss())
        
    def test(self, X):
        y = self.model.forward(X)
        return 1 if (np.squeeze(y) > 0.5).all() else 0
    
    def train(self, ittr = 100):
        for i in range(ittr):
            err = self.model._errs(self.X, self.y).sum()
            
            print(f"Loss in {i}th epoch: {err}")
            
            self.model.backward()
            
            for layer in self.model.layers:
                if isinstance(layer, Linear):
                    layer.update_w_b()
    

    def get_w(self):
        self.w = []
        self.b = []
        
        for layer in self.model.layers:
            if isinstance(layer, Linear):
                self.w.append(layer.w)
                self.b.append(layer.b)
        return self.w

    def s_model(self, f="model.w"):
        model_file = open(f, mode="wb")
        pickle.dump(self.get_w(), model_file)
        model_file.close()

    def l_model(self, f="model.w"):
        model_file = open(f, mode="rb")
        w = pickle.load(model_file)
        model_file.close()
        return w